In [1]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt


In [2]:
#select image and translational offset
path = r"3dTo2d\totalUncertancy\2_+00_+00_angle.png"
img = cv2.imread(path, 0)
dim, _ = img.shape[:2]
box_dim = LR.BOX_DIM
offset = 3
T_pospos = np.float32([[1,0,offset],[0,1,offset]])
T_posneg = np.float32([[1, 0, offset],[0, 1, -offset]])
T_negpos = np.float32([[1, 0, -offset],[0, 1, offset]])
T_negneg = np.float32([[1, 0, -offset],[0, 1, -offset]])
T_posnot = np.float32([[1, 0, offset], [0, 1, 0]])
T_negnot = np.float32([[1, 0, -offset], [0, 1, 0]])
T_notpos = np.float32([[1, 0, 0], [0, 1, offset]])
T_notneg = np.float32([[1, 0, 0], [0, 1, -offset]])
prim_rot = 0
secd_rot = 0


In [3]:
#translate image
#n-> negative N->nothing
img_pp = cv2.warpAffine(img, T_pospos, (dim, dim))
img_pn = cv2.warpAffine(img, T_posneg, (dim, dim))
img_np = cv2.warpAffine(img, T_negpos, (dim, dim))
img_nn = cv2.warpAffine(img, T_negneg, (dim, dim))
img_pN = cv2.warpAffine(img, T_posnot, (dim, dim))
img_nN = cv2.warpAffine(img, T_negnot, (dim, dim))
img_Np = cv2.warpAffine(img, T_notpos, (dim, dim))
img_Nn = cv2.warpAffine(img, T_notneg, (dim, dim))

"""plt.figure(figsize=(20,10))
plt.subplot(2,5,1)
plt.imshow(img, cmap="gray")
plt.title("Original")

plt.subplot(2, 5, 2)
plt.imshow(img_pp, cmap="gray")
plt.title("PosPos")

plt.subplot(2, 5, 3)
plt.imshow(img_pn, cmap="gray")
plt.title("PosNeg")

plt.subplot(2, 5, 4)
plt.imshow(img_np, cmap="gray")
plt.title("NegPos")

plt.subplot(2, 5, 5)
plt.imshow(img_nn,cmap="gray")
plt.title("NegNeg")

plt.subplot(2, 5, 6)
plt.imshow(img_pN, cmap="gray")
plt.title("PosNot")

plt.subplot(2, 5, 7)
plt.imshow(img_nN, cmap="gray")
plt.title("NegNot")

plt.subplot(2, 5, 8)
plt.imshow(img_Np, cmap="gray")
plt.title("NotPos")

plt.subplot(2, 5, 9)
plt.imshow(img_Nn, cmap="gray")
plt.title("NotNeg")

plt.show()"""


'plt.figure(figsize=(20,10))\nplt.subplot(2,5,1)\nplt.imshow(img, cmap="gray")\nplt.title("Original")\n\nplt.subplot(2, 5, 2)\nplt.imshow(img_pp, cmap="gray")\nplt.title("PosPos")\n\nplt.subplot(2, 5, 3)\nplt.imshow(img_pn, cmap="gray")\nplt.title("PosNeg")\n\nplt.subplot(2, 5, 4)\nplt.imshow(img_np, cmap="gray")\nplt.title("NegPos")\n\nplt.subplot(2, 5, 5)\nplt.imshow(img_nn,cmap="gray")\nplt.title("NegNeg")\n\nplt.subplot(2, 5, 6)\nplt.imshow(img_pN, cmap="gray")\nplt.title("PosNot")\n\nplt.subplot(2, 5, 7)\nplt.imshow(img_nN, cmap="gray")\nplt.title("NegNot")\n\nplt.subplot(2, 5, 8)\nplt.imshow(img_Np, cmap="gray")\nplt.title("NotPos")\n\nplt.subplot(2, 5, 9)\nplt.imshow(img_Nn, cmap="gray")\nplt.title("NotNeg")\n\nplt.show()'

In [4]:
#create back-projection for each image
imgs = [img, img_pp, img_pn, img_np, img_nn,img_pN,img_nN,img_Np,img_Nn]
boxes = []
flip = False

for i in imgs:
  line_coords = LR.createCoordsForLoadedImageFrom3d(i,flip,dim)
  world_coords = LR.performBresenham3D(line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  boxes.append(box)
  print("produced box")


produced box
produced box
produced box
produced box
produced box
produced box
produced box
produced box
produced box


In [5]:
print(len(boxes))

9


In [6]:
#add rotation to each box if wanted:
"""for i in range(len(boxes)):
  box_rot = LR.rotatePrimSecond(boxes[i],prim_rot,secd_rot)
  box_rot = np.where(box_rot>0.5,1,0)
  boxes[i] = box_rot
  print("Rotated box ", i)"""

'for i in range(len(boxes)):\n  box_rot = LR.rotatePrimSecond(boxes[i],prim_rot,secd_rot)\n  box_rot = np.where(box_rot>0.5,1,0)\n  boxes[i] = box_rot\n  print("Rotated box ", i)'

In [7]:
# Save box projections for each image as numpy arrays
name = "_-00_-00_"
imgs_name = ["img", "img_pp", "img_pn", "img_np",
             "img_nn", "img_px", "img_nx", "img_xp", "img_xn"] #numpy cannot saves name in lowercase

for i in range(len(boxes)):
  np.save(r"model_uncertainty\translated\offset_3\T" + name +
          str(offset) + "_" + imgs_name[i], boxes[i])


In [ ]:
#render box(es)
#x, y, z = LR.boxToWorldCorrds(boxes[0], box_dim)



In [ ]:
#mlab.points3d(x, y, z,
#              mode="cube", color=(1, 0, 0.5), scale_factor=1, opacity=1)
#mlab.show()
